In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import loading

%matplotlib inline

In [2]:
train_path = 'train/'
test = 'test1/'
import cv2

In [36]:
img_size = 128
classes = ['dogs', 'cats']
n_classes = 2
validation = 0.1
n_channels = 3
img_flat = n_channels * img_size * img_size

batch_size = 32
learning_rate = 1e-3


In [37]:
import glob
import cv2
import os

def read_images_in_folder(train_path):
    image_stack = []
    labels = []
    for img in glob.glob('train/*.jpg'): # All jpeg images
        label = img[img.find("/")+1:img.find(".")]
        labels.append(label)
        image = cv2.imread(img)
        img_resize = cv2.resize(image, (128, 128), cv2.INTER_LINEAR)
        image_stack.append(img_resize)
    return np.asarray(image_stack), labels
data, labels = read_images_in_folder(train_path)

KeyboardInterrupt: 

In [5]:
labels = [1 if label == 'dog' else 0 for label in labels]

In [6]:


#data = loading.read_data(train_path, img_size, classes, validation=validation)
#test_images, test_ids = loading.read_data_test(test_path, img_size)

In [7]:
labels = np.asarray(labels)

In [38]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(labels)))
print("- Test-set:\t\t{}".format((data.shape)))


Size of:
- Training-set:		25000
- Test-set:		(25000, 128, 128, 3)


In [ ]:
  w4 = weight_var(shape=[3, 3, 24, 64])
    b4 = bias_var([64])
    layer4 = tf.nn.conv2d(layer3, w4, strides=strides1, padding='SAME')
    layer4 = tf.nn.bias_add(layer4, b4)
    layer4 = tf.nn.relu(layer4)
    #print(layer3.get_shape)
    
    w5 = weight_var(shape=[3, 3, 64, 128])
    b5 = bias_var([128])
    layer5 = tf.nn.conv2d(layer4, w5, strides=strides1, padding='SAME')
    layer5 = tf.nn.bias_add(layer5, b5)
    layer5 = tf.nn.relu(layer5)

In [45]:
def weight_var(shape):
    w = tf.truncated_normal(shape=shape, stddev=0.5)
    return tf.Variable(w)

def bias_var(shape):
    b = tf.truncated_normal(shape=shape, stddev=0.5)
    return tf.Variable(b)

def CNN(x_):
    filter_ = 5
    strides1 = [1, 1, 1, 1]
    strides2 = [1, 2, 2, 1]
    ksize = strides2
    
    w1 = weight_var(shape=[3, 3, 3, 9])   
    b1 = bias_var([9])
    layer1 = tf.nn.conv2d(x_, w1, strides=strides1, padding='VALID')  #(, 44, 44, 64)
    layer1 = tf.nn.bias_add(layer1, b1)
    layer1 = tf.nn.relu(layer1)
    #print(layer1.get_shape)
    layer1 = tf.nn.max_pool(layer1, ksize=ksize, strides=strides2, padding='VALID') #(, 22, 22, 64)
    
    
    w2 = weight_var(shape=[3, 3, 9, 32])
    b2 = bias_var([32])
    layer2 = tf.nn.conv2d(layer1, w2, strides=strides1, padding='SAME') #(, 18, 18, 64)
    layer2 = tf.nn.bias_add(layer2, b2)
    #print(layer2.get_shape)
    
    layer2 = tf.nn.relu(layer2)
     
   
    layer2 = tf.nn.max_pool(layer2, ksize=ksize, strides=strides2, padding='SAME') #(, 9, 9, 64)
    #layer2 = tf.nn.dropout(layer2, 0.5)
    
    w3 = weight_var(shape=[3, 3, 32, 64])
    b3 = bias_var([64])
    layer3 = tf.nn.conv2d(layer2, w3, strides=strides1, padding='SAME')
    layer3 = tf.nn.bias_add(layer3, b3)
    layer3 = tf.nn.relu(layer3)

    
    layer5= tf.nn.max_pool(layer3, ksize=ksize, strides=strides2, padding='SAME')
    
    layer5 = tf.nn.dropout(layer5, 0.5)
    
    
    flatten = tf.contrib.layers.flatten(layer5)
    dim = flatten.get_shape().as_list()
    print(dim[1])
    
    wc1 = weight_var(shape=[(dim[1]), 512])
    b_c1 = bias_var([512])
    fc1 = tf.add(tf.matmul(flatten, wc1), b_c1)
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, 0.5)
    
    wc2 = weight_var(shape=[512, 128])
    b_c2 = bias_var([128])
    fc2 = tf.add(tf.matmul(fc1, wc2), b_c2)
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, 0.5)
    
    
    wc3 = weight_var(shape=[128, 16])
    b_c3 = bias_var([16])
    logits = tf.add(tf.matmul(fc2, wc3), b_c3)
    
    
    return logits
    

In [48]:
tf.reset_default_graph()
from sklearn.model_selection import train_test_split
X_tr, X_val,Y_tr, Y_val = train_test_split(data, labels, test_size = 0.1, random_state=0)



n_labels = len(set(labels))
feat = data.shape[1]
channels = data.shape[3]
batch_size = 64

x = tf.placeholder(dtype=tf.float32, shape=(None, feat, feat, channels))
#x_reshape = tf.reshape(x,[-1, img_size, img_size, n_channels] )

y = tf.placeholder(tf.int32, shape=(None))
y_hot = tf.one_hot(y, 2)

In [49]:
logits = CNN(x)
softmax = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_hot))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

prediction = tf.equal(tf.argmax(y_hot, 1), tf.argmax(softmax, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, dtype=tf.float32))


16384


In [50]:
def stats(X_batch, Y_batch):
    total_loss = 0
    total_acc = 0
    samples = len(X_batch)
    #X_batch = np.reshape(X_batch, [-1, n_steps, n_inputs])
    sess = tf.get_default_session()
    for offset in range(0, samples, batch_size):
        end = offset+batch_size
        xs_batch, ys_batch = X_batch[offset:end], Y_batch[offset:end]
        loss_, acc = sess.run([loss, accuracy], feed_dict={x:xs_batch, 
                                                           y:ys_batch
                                                           
                                                           })
        total_loss += (float(loss_)*xs_batch.shape[0])
        total_acc += (float(acc)*xs_batch.shape[0])

    return (total_loss)/samples, (total_acc)/samples

In [51]:
print(X_tr.shape, Y_tr.shape)

(22500, 128, 128, 3) (22500,)


In [52]:

acc_map = []

from sklearn.utils import shuffle

epochs = 200
samples = len(X_tr)

count = 0
print("Initializing Training")
try:
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        for epoch in range(epochs):
            X_tr, Y_tr = shuffle(X_tr, Y_tr)
            for offset in range(0, samples, batch_size):
                end = offset+batch_size
                xs_batch, ys_batch = X_tr[offset:end], Y_tr[offset:end]
                loss_, _ = sess.run([loss, optimizer], feed_dict={x: xs_batch, 
                                                                  y: ys_batch})    

            validation_loss, validation_acc = stats(X_val, Y_val)
            acc_map.append(round(validation_acc*100, 2))
            print("EPOCH {}".format(epoch + 1))
            print ("Validation Loss = {:.3f} and Validation Accuracy = {:.3f} %".format(validation_loss, 100 *validation_acc))
        plt.figure(figsize=(12, 5))
        plt.xticks(range(1, epochs+1))
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.grid(True)
        plt.plot(range(1, epochs+1), acc_map, '-o')
        for i, j in list(zip(range(1, epochs+1), acc_map))[0::3]:
            plt.text(i, j-1, str(j))
        #test_loss, test_acc = sess.run([loss, accuracy], feed_dict={x: xs_batch, 
                                                                    #y: ys_batch,
                                                                    #keep_prob:1.0})  
        print ("Test Loss {} and Test Accuracy {:.3f} %".format(test_loss, 100 *test_acc))
except KeyboardInterrupt:
    print('Training interrupted')

Initializing Training


InvalidArgumentError: logits and labels must be same size: logits_size=[64,16] labels_size=[64,2]
	 [[Node: SoftmaxCrossEntropyWithLogits = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape, Reshape_1)]]
	 [[Node: Mean/_19 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_735_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'SoftmaxCrossEntropyWithLogits', defined at:
  File "/home/cesar/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/cesar/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/cesar/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()